In [1]:
import warnings
warnings.simplefilter("ignore", UserWarning)

import pandas as pd

# Read data from files 
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
# Punkt tokenizer for sentence splitting
import nltk.data
 

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [4]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


In [5]:
# Check how many sentences we have in total - should be around 850,000+
print len(sentences)

795538


In [6]:
print sentences[0]

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']


In [7]:
print sentences[1]

[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-12-26 07:03:50,164 : INFO : 'pattern' package not found; tag filters are not available for English
2018-12-26 07:03:50,174 : INFO : collecting all words and their counts
2018-12-26 07:03:50,177 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-26 07:03:50,220 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-12-26 07:03:50,266 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-12-26 07:03:50,308 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-12-26 07:03:50,348 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types


Training model...


2018-12-26 07:03:50,398 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2018-12-26 07:03:50,441 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-12-26 07:03:50,487 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-12-26 07:03:50,529 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-12-26 07:03:50,573 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-12-26 07:03:50,618 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2018-12-26 07:03:50,667 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2018-12-26 07:03:50,740 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2018-12-26 07:03:50,783 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, keep

2018-12-26 07:03:53,651 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2018-12-26 07:03:53,696 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-12-26 07:03:53,742 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-12-26 07:03:53,769 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-12-26 07:03:53,772 : INFO : Loading a fresh vocabulary
2018-12-26 07:03:53,864 : INFO : effective_min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-12-26 07:03:53,867 : INFO : effective_min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018-12-26 07:03:53,927 : INFO : deleting the raw counts dictionary of 123504 items
2018-12-26 07:03:53,937 : INFO : sample=0.001 downsamples 48 most-common words
2018-12-26 07:03:53,940 : INFO : downsampling leaves estimated 1

2018-12-26 07:04:45,165 : INFO : EPOCH 4 - PROGRESS: at 24.09% examples, 759211 words/s, in_qsize 7, out_qsize 0
2018-12-26 07:04:46,174 : INFO : EPOCH 4 - PROGRESS: at 30.44% examples, 767512 words/s, in_qsize 8, out_qsize 0
2018-12-26 07:04:47,176 : INFO : EPOCH 4 - PROGRESS: at 36.68% examples, 770268 words/s, in_qsize 8, out_qsize 0
2018-12-26 07:04:48,181 : INFO : EPOCH 4 - PROGRESS: at 42.69% examples, 769916 words/s, in_qsize 8, out_qsize 0
2018-12-26 07:04:49,186 : INFO : EPOCH 4 - PROGRESS: at 49.10% examples, 775917 words/s, in_qsize 7, out_qsize 0
2018-12-26 07:04:50,188 : INFO : EPOCH 4 - PROGRESS: at 55.55% examples, 780813 words/s, in_qsize 7, out_qsize 0
2018-12-26 07:04:51,193 : INFO : EPOCH 4 - PROGRESS: at 61.92% examples, 784656 words/s, in_qsize 6, out_qsize 1
2018-12-26 07:04:52,203 : INFO : EPOCH 4 - PROGRESS: at 68.43% examples, 787741 words/s, in_qsize 7, out_qsize 0
2018-12-26 07:04:53,217 : INFO : EPOCH 4 - PROGRESS: at 73.68% examples, 777554 words/s, in_qsiz

In [9]:
model.doesnt_match("man woman child kitchen".split())

C:\Users\HoneyLab\AppData\Local\conda\conda\envs\ipykernel_py2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
C:\Users\HoneyLab\AppData\Local\conda\conda\envs\ipykernel_py2\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [10]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [12]:
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [13]:
model.wv.doesnt_match("paris berlin london austria".split())

'paris'

In [15]:
model.wv.most_similar("man")

[(u'woman', 0.6176740527153015),
 (u'lady', 0.5941395163536072),
 (u'lad', 0.5796729922294617),
 (u'farmer', 0.5346046090126038),
 (u'guy', 0.5274239778518677),
 (u'men', 0.5043917298316956),
 (u'gentleman', 0.49941501021385193),
 (u'monk', 0.49880602955818176),
 (u'millionaire', 0.4961532950401306),
 (u'person', 0.4944237172603607)]

In [18]:
model.wv.most_similar("queen")

[(u'princess', 0.6740788221359253),
 (u'bride', 0.6250922679901123),
 (u'belle', 0.604849100112915),
 (u'showgirl', 0.6033759117126465),
 (u'victoria', 0.5857219099998474),
 (u'stepmother', 0.5800154209136963),
 (u'maid', 0.578443169593811),
 (u'eva', 0.571523904800415),
 (u'duchess', 0.566586971282959),
 (u'nun', 0.5619477033615112)]

In [19]:
model.wv.most_similar("awful")

[(u'terrible', 0.7874165773391724),
 (u'atrocious', 0.7544002532958984),
 (u'horrible', 0.748379111289978),
 (u'dreadful', 0.7367974519729614),
 (u'abysmal', 0.7315496206283569),
 (u'horrendous', 0.725097119808197),
 (u'horrid', 0.6828507781028748),
 (u'appalling', 0.680413544178009),
 (u'amateurish', 0.6349622011184692),
 (u'lousy', 0.6284615993499756)]